In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# =========================================================
# Configuration
# =========================================================
files = {
    "Fluorescein": "Part 1 FL.xlsx",
    "Rhodamine B": "Part 1 RB.xlsx",
    "Rhodamine 6G": "Part 1 R6G.xlsx"
}

background_file = "Noise.xlsx"
EXPOSURE_CELL = "D1"

excluded_points = {
    "Fluorescein": [0.025],
    "Rhodamine B": [],
    "Rhodamine 6G": []
}

# =========================================================
# Utilities
# =========================================================
def cell_to_indices(cell):
    col = ord(cell[0].upper()) - ord('A')
    row = int(cell[1:]) - 1
    return row, col

exp_row, exp_col = cell_to_indices(EXPOSURE_CELL)

# =========================================================
# Model
# =========================================================
def fluorescence_model(c, A, B, delta):
    C = B + delta
    return A * (np.exp(-B * c) - np.exp(-C * c))

# =========================================================
# Data processing
# =========================================================
def integrate_sheet(filename, sheet, exp_row, exp_col, smoothing_window=5):
    df_raw = pd.read_excel(filename, sheet_name=sheet, header=None)
    exposure_time = df_raw.iat[exp_row, exp_col] / 100

    df = pd.read_excel(filename, sheet_name=sheet)
    wavelength = pd.to_numeric(df.iloc[:, 0], errors="coerce")
    intensity = pd.to_numeric(df.iloc[:, 1], errors="coerce")

    valid = wavelength.notna() & intensity.notna()
    wavelength = wavelength[valid]
    intensity = intensity[valid]

    mask = wavelength >= 470
    wavelength = wavelength[mask]
    intensity = intensity[mask]

    I_int = np.trapz(intensity, wavelength) / exposure_time

    intensity_smooth = pd.Series(intensity).rolling(
        window=smoothing_window, center=True, min_periods=1
    ).mean()

    deviations = np.abs(intensity - intensity_smooth)
    sigma_fluct = np.trapz(deviations, wavelength) / exposure_time / 10

    return I_int, sigma_fluct

def process_material(filename):
    xls = pd.ExcelFile(filename)

    c, I, sigma = [], [], []
    for sheet in xls.sheet_names:
        I_i, s_i = integrate_sheet(filename, sheet, exp_row, exp_col)
        c.append(float(sheet))
        I.append(I_i)
        sigma.append(s_i)

    c = np.array(c)
    I = np.array(I)
    sigma = np.array(sigma)

    order = np.argsort(c)
    return c[order], I[order], sigma[order]

# =========================================================
# Background
# =========================================================
bg_xls = pd.ExcelFile(background_file)
bg_sheet = bg_xls.sheet_names[0]
background_intensity, _ = integrate_sheet(background_file, bg_sheet, exp_row, exp_col)

print(f"Background intensity (normalized): {background_intensity:.4g}\n")

# =========================================================
# Plotting setup
# =========================================================
fig_main, ax_main = plt.subplots(figsize=(8, 5))
fig_res, ax_res = plt.subplots(
    nrows=3,
    ncols=1,
    figsize=(8, 7),
    sharex=True
)

# Map materials to axes
residual_axes = dict(zip(files.keys(), ax_res))


# =========================================================
# Main loop
# =========================================================
for material, file in files.items():
    c_all, I_all, sigma_all = process_material(file)

    I_bgsub = I_all - background_intensity

    excluded = np.isin(c_all, excluded_points.get(material, []))
    included = ~excluded

    c_fit = c_all[included]
    I_fit = I_bgsub[included]
    sigma = sigma_all[included]

    p0 = [max(I_fit), 0.1, 0.1]
    bounds = ([0, 0, 0], [np.inf, np.inf, np.inf])

    popt, pcov = curve_fit(
        fluorescence_model,
        c_fit,
        I_fit,
        p0=p0,
        bounds=bounds,
        sigma=sigma,
        absolute_sigma=True,
        maxfev=20000
    )

    residuals = I_fit - fluorescence_model(c_fit, *popt)
    chi2_red = np.sum((residuals / sigma) ** 2) / (len(I_fit) - len(popt))

    # Smooth curve
    c_smooth = np.linspace(c_fit.min(), c_fit.max(), 300)
    I_smooth = fluorescence_model(c_smooth, *popt)

    # ---- Main plot (error bars only)
    err_container = ax_main.errorbar(
        c_fit, I_fit, yerr=sigma,
        fmt=' ', capsize=3, label=material
    )

    color = err_container.lines[0].get_color()

    ax_main.plot(
        c_smooth, I_smooth,
        linestyle="--", linewidth=1,
        color=color
    )

    if np.any(excluded):
        ax_main.scatter(c_all[excluded], I_bgsub[excluded], facecolors='none', edgecolors='k')

    # ---- Residuals plot
    ax = residual_axes[material]

    ax.errorbar(
        c_fit,
        residuals,
        yerr=sigma,
        fmt=' ',
        capsize=3,
        color=color
    )

    ax.axhline(0, color='black', linewidth=1)
    ax.set_ylabel(f"{material}\nResiduals")
    ax.grid(True)


    A, B, delta = popt
    C = B + delta

    print(f"{material} fit parameters:")
    print(f"  A = {A:.4g}")
    print(f"  B = {B:.4g}")
    print(f"  C = {C:.4g}")
    print(f"  Reduced chi^2 = {chi2_red:.3f}\n")

# =========================================================
# Final formatting
# =========================================================
ax_main.set_xlabel("Concentration (mM)")
ax_main.set_ylabel("Intensity (arb. units)")
ax_main.set_title("Fluorescence vs. Concentration")
ax_main.legend()
ax_main.grid(True)
fig_main.tight_layout()

ax_res[-1].set_xlabel("Concentration (mM)")
fig_res.suptitle("Fit Residuals", y=0.98)
fig_res.tight_layout(rect=[0, 0, 1, 0.96])

plt.show()
